<a href="https://colab.research.google.com/github/yasirfaizahmed/Text-Classification-models/blob/main/models/text_sentimental_classification_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
! pip install datasets
! pip install --user -U nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00


In [7]:
from datasets import load_dataset

dataset = load_dataset("imsoumyaneel/sentiment-analysis-llama2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
import pandas as pd

dataset_df = pd.DataFrame(dataset['train'])

In [4]:
dataset_df.head()

,sentence,label,text
0,I'll throw out the garbage .,neutral,###Human:\nyou are a sentiment analist. guess ...
1,"So Dick , how about getting some coffee for to...",joy,###Human:\nyou are a sentiment analist. guess ...
2,"Come on , you can at least try a little , besi...",neutral,###Human:\nyou are a sentiment analist. guess ...
3,What ’ s wrong with that ? Cigarette is the th...,anger,###Human:\nyou are a sentiment analist. guess ...
4,"Not for me , Dick .",neutral,###Human:\nyou are a sentiment analist. guess ...


In [11]:
dataset_df['int_label'] = dataset_df['label'].map({'joy': 0, 'neutral': 1, 'sadness': 2, 'anger': 3, 'fear': 4, 'love': 5, 'surprise': 6})
dataset_df.head()


,sentence,label,text,int_label
0,I'll throw out the garbage .,neutral,###Human:\nyou are a sentiment analist. guess ...,1
1,"So Dick , how about getting some coffee for to...",joy,###Human:\nyou are a sentiment analist. guess ...,0
2,"Come on , you can at least try a little , besi...",neutral,###Human:\nyou are a sentiment analist. guess ...,1
3,What ’ s wrong with that ? Cigarette is the th...,anger,###Human:\nyou are a sentiment analist. guess ...,3
4,"Not for me , Dick .",neutral,###Human:\nyou are a sentiment analist. guess ...,1


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

